In [1]:
#Main library
import tensorflow as tf

#Helper libraries
import os, pickle, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dir = ""
checkpoint_dir = dir + "model1/"
class_names = ['apple', 'apricot', 'avocado', 'banana', 'bell pepper', 'black berry', 'blueberry', 'cantaloupe', 'cherry', 
               'coconut', 'coffee', 'desert fig', 'eggplant', 'fig', 'grape', 'grapefruit', 'kiwi', 'lemon', 'lime', 'lychee', 
               'mango', 'orange', 'pear', 'pineapple', 'pomegranate', 'pumpkin', 'raspberry', 'strawberry', 'watermelon']

In [3]:
def load_model(dir):
  model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(29)
  ])

  model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

  latest = tf.train.latest_checkpoint(dir)
  model.load_weights(latest).expect_partial()
  return model

In [8]:
def predict(path, model):
  img = tf.keras.preprocessing.image.load_img(path, target_size=(180, 180))
  img_array = tf.keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)
  predictions = model.predict(img_array)
  score = tf.nn.softmax(predictions[0])
  return score

In [5]:
model = load_model(checkpoint_dir)

In [11]:
model.build((1, 180, 180, 3))
model.save("fig.h5")

In [12]:
model = tf.keras.models.load_model("fig.h5")

In [13]:
img_path = input("path = ")
prediction = predict(img_path, model)

confidences = {class_names[i]: confidence.numpy() * 100 for i, confidence in enumerate(prediction)}
print(confidences)

(1, 180, 180, 3)
{'apple': 1.1283261701464653, 'apricot': 0.0028274002033867873, 'avocado': 0.005404209514381364, 'banana': 24.82135444879532, 'bell pepper': 0.0017651822417974472, 'black berry': 0.00047797684601391666, 'blueberry': 0.00012341570254648104, 'cantaloupe': 5.963262661445867e-08, 'cherry': 0.02399351360509172, 'coconut': 0.04860591725446284, 'coffee': 2.687416411936283, 'desert fig': 23.966917395591736, 'eggplant': 0.030564621556550264, 'fig': 16.56009703874588, 'grape': 13.561521470546722, 'grapefruit': 4.3643806129693985, 'kiwi': 0.01851980050560087, 'lemon': 0.001379878267471213, 'lime': 0.00012554238537632045, 'lychee': 1.9089221954345703, 'mango': 1.0279372334480286, 'orange': 2.4051354685639126e-06, 'pear': 0.1553662121295929, 'pineapple': 0.012668212002608925, 'pomegranate': 9.638558328151703, 'pumpkin': 8.282031949136126e-07, 'raspberry': 0.03241519443690777, 'strawberry': 9.189555498778645e-05, 'watermelon': 0.00023915017663966864}


In [14]:
print("\nGot {} with {}% confidence.".format(class_names[np.argmax(prediction)], max(prediction) * 100))


Got banana with 24.821353912353516% confidence.


In [36]:

figs = confidences["fig"] + confidences["desert fig"]

if "fig" in predicted_class:
    predicted_class = "FIG"
    confidence = figs
else:
    confidence = confidences[class_names.index(predicted_class)] - figs
    predicted_class = "NOT FIG"

print("Got {} with {}% confidence.".format(predicted_class, str(confidence * 100)[:5]))

Got NOT FIG with 84.96% confidence.
